In [ ]:
!pip install openpyxl

from datetime import date
import pandas as pd
import requests
import json

In [ ]:
# Read the transactions file
transactions = pd.read_csv("transactions.csv")
transactions.head()

In [ ]:
# Convert column1 from "MMM-dd-yyyy" to date
transactions["AcquiredDate"] = pd.to_datetime(transactions["Date acquired"], format="%b-%d-%Y")
transactions["SoldDate"] = pd.to_datetime(transactions["Date sold"], format="%b-%d-%Y")
transactions.head()

In [ ]:
# convert Quantity to float, floor it, then convert to int
transactions["Quantity"] = transactions["Quantity"].astype(float).floordiv(1).astype(int)
transactions.head()

In [ ]:
# Convert Cost basis from string with $ and commas to float
transactions["CostUSD"] = transactions["Cost basis"].replace("[\$,]", "", regex=True).astype(float)

# Same for Proceeds
transactions["ProceedsUSD"] = transactions["Proceeds"].replace("[\$,]", "", regex=True).astype(float)

# Just keep Gain/loss for reference, we won't use it
transactions["Gain/loss"] = transactions["Gain/loss"].replace("[\$,]", "", regex=True).astype(float)

transactions.head()

In [ ]:
# Sum Gain/loss in USD
sum_gain_loss = transactions['Gain/loss'].sum()
print(f"Sum Gain/Loss in USD: {sum_gain_loss}")

In [ ]:
def lookup_exchange_rate(lookup_date):
    fdate = lookup_date.strftime("%Y-%m-%d")
    url = f"https://data.norges-bank.no/api/data/EXR/B.USD.NOK.SP?startPeriod={fdate}&endPeriod={fdate}&format=sdmx-json&locale=no"
    
    response = requests.get(url)
    data = response.json()

    return float(data['data']['dataSets'][0]['series']['0:0:0:0']['observations']['0'][0])

In [ ]:
d = date(2022, 12, 30)
lookup_exchange_rate(d)

In [ ]:
# Create a new column AcquiredExchangeRate and fill it with values from lookup_exchange_rate given the AcquiredDate column
transactions["AcquiredExchangeRate"] = transactions["AcquiredDate"].apply(lookup_exchange_rate)

# Same for SoldDate
transactions["SoldExchangeRate"] = transactions["SoldDate"].apply(lookup_exchange_rate)

transactions.head()

In [ ]:
# Calculate CostNOK
transactions["CostNOK"] = transactions["CostUSD"] * transactions["AcquiredExchangeRate"]

# Calculate ProceedsNOK
transactions["ProceedsNOK"] = transactions["ProceedsUSD"] * transactions["SoldExchangeRate"]

# Calculate GainLossNOK
transactions["GainLossNOK"] = transactions["ProceedsNOK"] - transactions["CostNOK"]

transactions.head()

In [ ]:
# Sum GainLossNOK
total_gain_loss = transactions["GainLossNOK"].sum()
total_gain_loss

In [ ]:
# Calculate tax at 31%
total_tax = total_gain_loss * 0.31
total_tax

In [ ]:
# Save the documentation as Excel file for submission to tax authorities
transactions.to_excel("transactions_processed.xlsx")